In [1]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("TickTock")

    Updating registry at `/opt/julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.7/Project.toml`
  No Changes to `/opt/julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.7/Project.toml`
  No Changes to `/opt/julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.7/Project.toml`
  No Changes to `/opt/julia/environments/v1.7/Manifest.toml`


In [2]:
using Random
using DataFrames
using Statistics
using CSV
using TickTock

In [3]:
function get_instancia(n)
    return rand(1:100, n)
end

function get_permutacoes(instancia)
    return [shuffle(instancia) for _=1:20]
end

get_permutacoes (generic function with 1 method)

In [4]:
function get_custo(instancia, tamanho, custo_total)
    if tamanho == 1
        return instancia[tamanho], 0
    else
        espera, custo_total = get_custo(instancia[1:tamanho-1], tamanho-1, custo_total)
        custo = instancia[tamanho] + espera
        return custo, custo_total + custo
    end
end

function get_vizinhos_one_flip(solucao, tamanho)
    vizinhos = Vector()
    for indice in 1:tamanho-1
        vizinho = copy(solucao)
        vizinho[indice], vizinho[indice+1] = solucao[indice+1], solucao[indice]
        push!(vizinhos, vizinho)
    end
    vizinho = copy(solucao)
    vizinho[1], vizinho[tamanho] = solucao[tamanho], solucao[1]
    push!(vizinhos, vizinho)
    return vizinhos
end

get_vizinhos_one_flip (generic function with 1 method)

In [5]:
function get_melhor_vizinho(vizinhos, is_first_improved, menor_custo, melhor_solucao, tamanho)
    for vizinho in vizinhos
        _, custo = get_custo(vizinho, tamanho, 0)
        if custo < menor_custo
            menor_custo = custo
            melhor_solucao = vizinho
            if is_first_improved
                break
            end
        end
    end
    return melhor_solucao, menor_custo
end

get_melhor_vizinho (generic function with 1 method)

### 2-a

In [6]:
resultados = []
for is_first_improved in [true, false]
    if is_first_improved
        alg = "FI"
    else
        alg = "BI"
    end
    for n in [100, 200, 300, 400, 500]
        tick()
        instancia_inicial = get_instancia(n)
        permutacoes_iniciais = get_permutacoes(instancia_inicial)
        solucao_inicial = Nothing
        menor_custo = 99999999999
        solucao_inicial, menor_custo = get_melhor_vizinho(permutacoes_iniciais, is_first_improved, 
                                        menor_custo, solucao_inicial, n)
        solucao = solucao_inicial
        for iteracao in 1:100
            vizinhos = get_vizinhos_one_flip(solucao, n)
            solucao, menor_custo = get_melhor_vizinho(vizinhos, is_first_improved, menor_custo, solucao, n)
        end
        tempo = tok()
        push!(resultados, tuple(alg, n, 20, round(tempo, digits=2), "NA", menor_custo))
    end
end

┌ Info:  started timer at: 2022-03-02T19:09:10.517
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-03-02T19:09:10.758
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-03-02T19:09:10.978
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-03-02T19:09:11.337
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-03-02T19:09:11.735
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-03-02T19:09:12.377
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-03-02T19:09:12.724
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-03-02T19:09:15.620
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-03-02T19:09:23.700
└ @ T

In [7]:
df = DataFrame(NamedTuple{(:alg, :n, :no, :time, :iterations, :value)}.(resultados))

,alg,n,no,time,iterations,value
,String,Int64,Int64,Float64,String,Int64
1,FI,100,20,0.25,NA,270359
2,FI,200,20,0.22,NA,1037021
3,FI,300,20,0.36,NA,2191434
4,FI,400,20,0.4,NA,3787486
5,FI,500,20,0.64,NA,6471495
6,BI,100,20,0.35,NA,211622
7,BI,200,20,2.9,NA,917386
8,BI,300,20,8.08,NA,2153319
9,BI,400,20,11.68,NA,3913652


In [8]:
CSV.write("resultados_tarefa2.csv", df)

"resultados_tarefa2.csv"

#### Média e desvio padrão de tempo e custo em função da estratégia de busca local

In [9]:
combine(groupby(df, "alg"), [:time => mean, :time => std, :value => mean, :value => std])

,alg,time_mean,time_std,value_mean,value_std
,String,Float64,Float64,Float64,Float64
1,FI,0.374,0.166373,2.75156e6,2.46581e6
2,BI,8.15,6.94328,2.65909e6,2.38168e6
